# 以CLIP與顯著圖來實作多模態問答


## CLIP與顯著圖



- CLIP（對比語言-圖像預訓練，Contrastive Language–Image Pre-training）是一個可以同時處理圖像和文本的神經網絡。它經過訓練來預測隨機文本片段是否能準確描述某張圖像。
- 顯著性圖是一種視覺化技術，用於突出顯示圖像中的關鍵區域。例如，它可以用來解釋圖像分類的預測對於特定標籤的意義。

## 安裝所需套件

In [ ]:
%pip install -q "openvino>=2023.1.0"
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu transformers torch gradio

## 匯入所需套件

In [ ]:
from pathlib import Path
from typing import Tuple, Union, Optional
from urllib.request import urlretrieve

from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import requests
import torch
import tqdm
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

## CLIP模型


- 使用 OpenAI 提供的 CLIP 模型版本 clip-vit-base-patch16，這是一個基於 Vision Transformer（ViT）的 CLIP 模型。

In [ ]:
# 初始化CLIP模型與處理器
# CLIP模型由OpenAI開發，用於文本與圖像的多模態學習
model_checkpoint = "openai/clip-vit-base-patch16"

model = CLIPModel.from_pretrained(model_checkpoint).eval()
processor = CLIPProcessor.from_pretrained(model_checkpoint)

## 定義函式



- get_random_crop_params 生成隨機裁剪的座標與尺寸。
- get_crop_image 獲取實際裁剪的圖像部分
- update_saliency_map 根據計算出的相似度更新顯著性圖。
- cosine_similarity 函數則是上述餘弦相似度公式的實作。

In [ ]:
def get_random_crop_params(
    image_height: int, image_width: int, min_crop_size: int
) -> Tuple[int, int, int, int]:
    crop_size = np.random.randint(min_crop_size, min(image_height, image_width))
    x = np.random.randint(image_width - crop_size + 1)
    y = np.random.randint(image_height - crop_size + 1)
    return x, y, crop_size


def get_cropped_image(
    im_tensor: np.array, x: int, y: int, crop_size: int
) -> np.array:
    return im_tensor[
        y : y + crop_size,
        x : x + crop_size,
        ...
    ]


def update_saliency_map(
    saliency_map: np.array, similarity: float, x: int, y: int, crop_size: int
) -> None:
    saliency_map[
        y : y + crop_size,
        x : x + crop_size,
    ] += similarity


def cosine_similarity(
    one: Union[np.ndarray, torch.Tensor], other: Union[np.ndarray, torch.Tensor]
) -> Union[np.ndarray, torch.Tensor]:
    return one @ other.T / (np.linalg.norm(one) * np.linalg.norm(other))

## 設定圖片及查詢提示詞


- 先將圖片上傳至左側資料夾
- 修改程式中的問題描述
- 再修改程式中的圖檔名稱
- 最後執行程式格

In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np

# 定義參數
n_iters = 300  # 迭代次數，用於生成顯著圖
min_crop_size = 50  # 最小裁剪尺寸

# 問題文字
query = "Write your question here."

# 使用本地圖片
image_path = Path("/content/圖片名稱.jpg")
image = Image.open(image_path)
im_tensor = np.array(image)

# 圖片尺寸
x_dim, y_dim = image.size

print(f"Image size: {x_dim}x{y_dim}")

## 文本及圖像預處理像

In [ ]:
# 使用CLIP處理器將圖像和文本轉換為模型輸入格式
inputs = processor(text=[query], images=[im_tensor], return_tensors="pt")
with torch.no_grad():
    results = model(**inputs)
results.keys()

## 計算相似度

In [ ]:
# 計算初始的查詢文本與整體圖像的相似度
initial_similarity = cosine_similarity(results.text_embeds, results.image_embeds).item()
# 初始化顯著性圖 (saliency map)，其大小與圖像相同
saliency_map = np.zeros((y_dim, x_dim))

# 設定程序的迭代次數
for _ in tqdm.notebook.tqdm(range(n_iters)):
    # 隨機生成圖像裁剪的參數，包括裁剪起點 (x, y) 和裁剪大小
    x, y, crop_size = get_random_crop_params(y_dim, x_dim, min_crop_size)
    # 根據生成的參數裁剪圖像
    im_crop = get_cropped_image(im_tensor, x, y, crop_size)

    # 使用 CLIP 處理器對裁剪的圖像和查詢文本進行預處理，轉換為模型輸入格式
    inputs = processor(text=[query], images=[im_crop], return_tensors="pt")
    with torch.no_grad():
        # 將預處理的輸入傳入模型，計算裁剪圖像與查詢文本的相似度
        results = model(**inputs)

    # 計算裁剪圖像的相似度與初始相似度的差異
    similarity = cosine_similarity(results.text_embeds, results.image_embeds).item() - initial_similarity
    # 使用計算出的相似度更新顯著性圖中對應的區域
    update_saliency_map(saliency_map, similarity, x, y, crop_size)


## 繪製顯著圖

In [ ]:
plt.figure(dpi=150)
plt.imshow(saliency_map, norm=colors.TwoSlopeNorm(vcenter=0), cmap='jet')
plt.colorbar(location="bottom")
plt.title(f'Query: \"{query}\"')
plt.axis("off")
plt.show()

## 叠合原圖與顯著圖

In [ ]:
def plot_saliency_map(image_tensor: np.ndarray, saliency_map: np.ndarray, query: Optional[str]) -> None:
    # 設置圖像的 DPI（解析度）
    fig = plt.figure(dpi=150)

    # 顯示原始圖像
    plt.imshow(image_tensor)

    # 疊加顯著性圖
    plt.imshow(
        saliency_map,
        norm=colors.TwoSlopeNorm(vcenter=0),  # 設置顯著性圖的標準化範圍，以 0 為中心
        cmap="jet",  # 使用 "jet" 色彩映射，顯示不同的顏色區分強度
        alpha=0.5,  # 設置透明度，確保可以同時看到原始圖像和顯著性圖
    )

    # 如果有查詢文本，將其作為標題顯示
    if query:
        plt.title(f'Query: "{query}"')

    # 關閉圖像的座標軸顯示
    plt.axis("off")

    plt.show()
    #return fig  # 返回生成的圖像對象

# 調用函數以顯示顯著性圖
plot_saliency_map(im_tensor, saliency_map, query)
